In [3]:
%run Jansen_And_Rit.py
%run OptimiseFunctions.py

In [ ]:
# Plot this for the optimal params resulting FC matrix 
fig = plt.figure(figsize=(8, 6))
plt.imshow(FC_matrix, cmap='RdBu', vmin=-1, vmax=1)
plt.colorbar()
plt.title('Correlation Matrix')
plt.xlabel('Node Index')
plt.ylabel('Node Index')
plt.show()
plt.close(fig)